In [1]:
from collections import deque
from typing import List
import random
import string
from queue import PriorityQueue
from enum import Enum
from dataclasses import dataclass

# Задание 1 (4 балла)

## Обратный порядок слов в блоках текста

Дан текстовый файл, каждое предложение которого занимает одну строку. Напишите программу, которая разделяет текст на блоки — каждый блок состоит из нескольких предложений. Затем переворачивает порядок слов только внутри каждого предложения, не меняя порядок самих предложений в блоке.

Файл содержит:

* Привет как дела
* На улице идет дождь
* Я люблю программирование

Критерии оценки:

* Текст разделен на блоки корректно — 2 балла.
* Корректно перевернуты блоки — 2 балла.

In [2]:
# подготовка файла
with open('input.txt', 'w', encoding='utf-8') as f:
    f.write('Привет как дела\nНа улице идет дождь\nЯ люблю программирование')

In [3]:
def split_into_blocks(lines: List[str], block_size: int) -> List[str]:
    # разделение на блоки равного размера
    blocks = []
    for i in range(0, len(lines), block_size):
        blocks.append(lines[i:i + block_size])
    return blocks

def reverse_words(blocks: List[List[str]]) -> List[List[str]]:
    # переворачивание порядка слов внутри каждого предложения
    reversed_blocks = []
    for block in blocks:
        reversed_block = [' '.join(b.strip().split()[::-1]) for b in block]
        reversed_blocks.append(reversed_block)
    return reversed_blocks

# чтение файла
with open('input.txt', 'r', encoding='utf-8') as input_file:
    lines = input_file.readlines()

# разделение на блоки с размером 2
blocks = split_into_blocks(lines, block_size=2)
# переворацивание пордка слов
reversed_blocks = reverse_words(blocks)

# вывод результатов
print("Блоки, в которых перевернут порядок слов:", *reversed_blocks, sep='\n')

Блоки, в которых перевернут порядок слов:
['дела как Привет', 'дождь идет улице На']
['программирование люблю Я']


# Задание 2 (2 балла)

Напишите функцию, которая принимает строку и сжимает её определённым образом. Строки содержат произвольные символы, включая пробелы и спецсимволы, и требуют точного учета длины при кодировании. Сжатие строки происходит сериями одинаковых символов в формате символколичество, но только если длина сжатой строки не превышает исходную.

Примеры:

* вход: aaabbc → вывод: a3b2c,
* вход: abcd → вывод: abcd (так как сжатая равна).

Критерии оценки:

* Корректно очищен текст — 1 балл.
* Выполняется условие — 1 балл.

In [4]:
def compress(line: str) -> str:
    result = ''
    counter = 0
    current_sym = line[0]
    
    for sym in line:
        if current_sym == sym:
            counter += 1
        else:
            result += current_sym + str(counter) * (counter != 1)
            counter = 1
            current_sym = sym
            
    result += current_sym + str(counter) * (counter != 1)

    return line if len(line) < len(result) else result

print(compress('aaabbc'))
print(compress('abcd'))

a3b2c
abcd


# Задание 3 (4 балла)
Хаотичные скобки

Реализуйте функцию, которая проверяет, правильно ли расставлены скобки в строке (включая круглые, квадратные и фигурные скобки).

Критерии оценки:

* Функция выполняет условие — 4 балла.

In [5]:
def check_brackets(line: str) -> bool:
    open_br = '[({'
    close_br = '])}'
    open_close = {'[': ']', '(': ')', '{': '}'}
    stack = deque()
    
    for sym in line:
        if sym in open_br:
            stack.append(sym)
        elif sym in close_br:
            if len(stack) == 0:
                return False
            last_br = stack.pop()
            if open_close[last_br] != sym:
                return False
    return True

print(check_brackets('()[]{}'))
print(check_brackets('(())[]{}'))
print(check_brackets('())[]{}'))
print(check_brackets('([)]{}'))
print(check_brackets('([{)]}'))

True
True
False
False
False


# Задание 4 (4 балла)
## Генератор случайных паролей

Напишите функцию, которая генерирует пароль заданной длины. В реализации надо учитывать, что:

* Пароль должен содержать буквы, цифры и специальные символы.
* Длина пароля задается пользователем.

Критерии оценки:

* Функция выполняет все заявленные условия — 4 балла.


In [6]:
def generate_pass(length: int = 3) -> str:
    if length < 3:
        raise ValueError('Длина пароля не может быть меньше 3')
        
    letters = string.ascii_letters
    digits = string.digits
    special = string.punctuation
    all_sym = letters + digits + special

    # хотя-бы один символ каждого типа должен присутствовать
    password = [
        random.choice(letters),
        random.choice(digits),
        random.choice(special)
    ]

    # добавляем оставшиеся символы и перемешиваем список
    password += random.choices(all_sym, k=length-3)
    random.shuffle(password)

    return ''.join(password)

for length in range(3, 10):
    print(generate_pass(length))

Y&9
"q7D
s>l7-
$fE1,t
Ku^!6fo
9S'hCQ8z
}kgjEC*@3


# Задание 5 (6 баллов)
Эмуляция работы электронной очереди

Напишите класс для симуляции работы электронной очереди, например, в банке.

Критерии оценки:

* Система должна поддерживать добавление клиентов с указанием их приоритета (например, VIP, обычный) — 1 балл.
* Выбор клиента для следующей обработки должен учитывать приоритет — 1 балл.
* Реализуйте отчет для администрации с информацией, сколько времени заняло обслуживание клиентов — 4 балла.


In [7]:
# приоритеты клиентов
class Priority(Enum):
    VIP = 1
    COMMON = 2


@dataclass
class Client:
    id: int
    priority: Priority
    start_service_time: int # время начала обслуживания
    service_time: int # длительность обслуживания


class BankQueue:
    def __init__(self):
        self.queue = PriorityQueue() # очередь с реализованным приоритетом
        self.windows = {1: None, 2: None} # окна обслуживания
        self.id_counter = 1 # счетчик id клиентов
        self.current_time = 1 # текущее время в минутах

    def add_client(self, priority):
        # создание клиента со случайным временем обслуживания
        client = Client(self.id_counter, priority, 0, random.randint(1, 30))
        self.id_counter += 1
        self.queue.put((priority.value, self.id_counter, client))

    def process(self):
        # пока есть клиенты в очереди или хотя бы одно окно занято
        while (not self.queue.empty()) or any(self.windows.values()):
            # освобождаем окна где закончилось обслуживание
            for window, client in zip(self.windows.keys(), self.windows.values()):
                if client and client.start_service_time + client.service_time <= self.current_time:
                    self.windows[window] = None
                    print(f'Конец обслуживания клиента {client.id}\nВремя: {self.current_time}\nДлительность обслуживания: {client.service_time}\n' + '-' * 20)

            # отправка клиентов к свободным окнам
            while (not self.queue.empty()) and not all(self.windows.values()):
                client = self.queue.get()[2]
                empty_window = next(w for w, c in self.windows.items() if c is None)
                client.start_service_time = self.current_time
                self.windows[empty_window] = client
                print(f'Начало обслуживания клиента {client.id}\nВремя: {self.current_time}\n' + '-' * 20)

            self.current_time += 1


# Пример использования
queue = BankQueue()

queue.add_client(Priority.COMMON)
queue.add_client(Priority.COMMON)
queue.add_client(Priority.VIP)
queue.add_client(Priority.COMMON)

print('Обслуживание клиентов\n' + '-' * 20)
queue.process()


Обслуживание клиентов
--------------------
Начало обслуживания клиента 3
Время: 1
--------------------
Начало обслуживания клиента 1
Время: 1
--------------------
Конец обслуживания клиента 3
Время: 20
Длительность обслуживания: 19
--------------------
Начало обслуживания клиента 2
Время: 20
--------------------
Конец обслуживания клиента 1
Время: 30
Длительность обслуживания: 29
--------------------
Начало обслуживания клиента 4
Время: 30
--------------------
Конец обслуживания клиента 4
Время: 41
Длительность обслуживания: 11
--------------------
Конец обслуживания клиента 2
Время: 45
Длительность обслуживания: 25
--------------------


# Задание 6 (2 балла)
Проверка на «почти палиндром»

Напишите программу, которая проверяет, является ли строка палиндромом или «почти палиндромом». «Почти палиндром» означает, что можно удалить одну букву, чтобы строка стала палиндромом.

Критерии оценки:

* Функция выполняет все заявленные условия — 2 балла.


In [8]:
def is_pal(line, deleted_count=0):
    # два указателя
    l, r = 0, len(line) - 1
    while l < r:
        if line[l] == line[r]:
            l += 1
            r -= 1
        else:
            # если еще не удаляли букв - рассматриваем два варианта с удалением
            if deleted_count < 1:
                return is_pal(line[l: r], deleted_count + 1) or is_pal(line[l + 1: r + 1], deleted_count + 1)
            return False
    return True

print(is_pal('aba'))
print(is_pal('abaa'))
print(is_pal('abaaa'))
print(is_pal('abbaa'))
print(is_pal('abbaaa'))

True
True
True
True
False


# Задание 7 (9 баллов)
## Задача Шредингера

Разработайте программу, которая «стирает» фрагменты текста в файле. Например:

1. Пользователь указывает файл и процент текста, который нужно удалить (например, 30%).
2. Программа случайно выбирает слова или части абзацев и заменяет их на пробел или ..., сохраняя общий объем документа.

Вход:
«Сегодня солнечный день, и я собираюсь гулять в парке с моими друзьями.»

Вывод (удалено ~30%):
«Сегодня ... день, и я собираюсь гулять ... моими друзьями.»

Критерии оценки:

* Чтение файла и ввод параметров пользователя — 3 балла.
* Реализация алгоритма случайного удаления текста — 5 баллов.
* Запись результата и вывод пользователю — 1 балл.


In [9]:
# подготовка файла
text = '''Он благополучно избегнул встречи с своею хозяйкой на лестнице.
Каморка его приходилась под самою кровлей высокого пятиэтажного дома и походила более
на шкаф, чем на квартиру. Квартирная же хозяйка его, у которой он
нанимал эту каморку с обедом и прислугой, помещалась одною лестницей ниже,
в отдельной квартире, и каждый раз, при выходе на улицу, ему непременно надо
было проходить мимо хозяйкиной кухни, почти всегда настежь отворенной на лестницу.
И каждый раз молодой человек, проходя мимо, чувствовал какое-то болезненное и трусливое ощущение,
которого стыдился и от которого морщился. Он был должен кругом хозяйке и боялся с нею встретиться.'''

with open('input.txt', 'w', encoding='utf-8') as f:
    f.write(text)

In [10]:
def get_strip_parts(s: str, chars: str = None) -> tuple:
    # сохранение пункцтуации перед удалением слова
    stripped = s.strip(chars)
    left = s[:len(s) - len(s.lstrip(chars))]
    right = s[len(stripped) + len(left):]
    return left, stripped, right


def shred(lines, percent):
    # разделение строк на слова и выбор слов для удаления
    lines_words = [line.split() for line in lines]
    words_count = sum(map(len, lines_words))
    delete_count = int(words_count * percent / 100)
    word_indexes_to_delete = random.sample(list(range(0, words_count)), k=delete_count)
    current_word_index = 0

    # проход по всем словам с удалением выбранных ранее слов
    for line_ind in range(len(lines_words)):
        for word_ind in range(len(lines_words[line_ind])):
            if current_word_index in word_indexes_to_delete:
                # замена слова на ... с сохранением пунктуации
                left, word, right = get_strip_parts(lines_words[line_ind][word_ind], string.punctuation)
                lines_words[line_ind][word_ind] = left + '...' + right
            current_word_index += 1
    return [' '.join(lines_words[line_ind]) for line_ind in range(len(lines_words))]

# чтение файла
with open('input.txt', 'r', encoding='utf-8') as input_file:
    lines = input_file.readlines()
text = '\n'.join(lines)

percent = 30
print(f'Исходный текст:\n{text}\n' + '-' * 100)
new_text = "\n".join(shred(text.split("\n"), percent))
print(f'Результат (удалено ~{percent}%):\n{new_text}')

Исходный текст:
Он благополучно избегнул встречи с своею хозяйкой на лестнице.

Каморка его приходилась под самою кровлей высокого пятиэтажного дома и походила более

на шкаф, чем на квартиру. Квартирная же хозяйка его, у которой он

нанимал эту каморку с обедом и прислугой, помещалась одною лестницей ниже,

в отдельной квартире, и каждый раз, при выходе на улицу, ему непременно надо

было проходить мимо хозяйкиной кухни, почти всегда настежь отворенной на лестницу.

И каждый раз молодой человек, проходя мимо, чувствовал какое-то болезненное и трусливое ощущение,

которого стыдился и от которого морщился. Он был должен кругом хозяйке и боялся с нею встретиться.
----------------------------------------------------------------------------------------------------
Результат (удалено ~30%):
Он благополучно избегнул встречи с своею ... на лестнице.

Каморка его приходилась под самою кровлей ... пятиэтажного дома и ... более

на шкаф, чем ... квартиру. Квартирная же хозяйка ..., у ... ...

..